In [1]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "loGu88Z",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "u6p1OlcKS8K0ulvvGBdVKw",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['localhost', 'otherhost'],
    http_auth=('elastic', 'changeme'),
    port=9200
)

In [3]:
import os
print "current dir: ", os.getcwd()
os.chdir("/home/foodmap/food101/")
print "current dir: ", os.getcwd()

current dir:  /home/foodmap/food101/notebooks
current dir:  /home/foodmap/food101


In [4]:
from location.locations import Cities, Countries
from twitter.Tweet import Tweet
from location.get_location_from_tweet import *
import random

citiesIndex, citiesInfo = Cities.loadFromFile()
countriesIndex, countriesInfo = Countries.loadFromFile()
ccDict = Countries.countryCodeDict(countriesInfo)

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [5]:
inputFile = "/home/foodmap/food-tweets-2017-04-23.json.gz"

In [ ]:
def get_media_url(tweet):
    if "media" in tweet["entities"]:
        foundMedia = tweet["entities"]["media"]
        return foundMedia[0]["media_url"]
    else:
        return None

In [ ]:
def get_food_category(img_url):
    categ_list = ["Pizza", "Pasta", "Sushi", None]
    return random.choice(categ_list)

In [ ]:
a = getLocationsFromToken("austin", citiesIndex, citiesInfo, countriesIndex, countriesInfo)
print a
print citiesIndex["london"]
print citiesInfo[5809844]

cities, countries = getUserLocation("London, England", citiesIndex, citiesInfo, countriesIndex, countriesInfo)
print cities, countries

In [ ]:
import json, time
tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
countImg = 0
countCategory = 0
countPlace = 0


for tweet in tweetsAsDict:
    i+=1
    hasImg = False
    hasCategory = False
    hasCountry = False

    # 1. has photo?
    if get_media_url(tweet) is not None:
        hasImg = True
        countImg += 1
    else:
        continue

    # 2. has food category?
    if get_food_category(tweet) is not None:
        hasCategory = True
        countCategory += 1
    else:
        continue

    # 3. has location? we need city and country; record 2 separate fields
    ## 3.1. check for place
    tweet_coords, tweet_place_city, tweet_place_country, tweet_place_country_code, user_loc = getLocationData(tweet)
    ## 3.2. check for user location
    if (tweet_place_country is None):
        user_cities, user_countries = getUserLocation(user_loc, citiesIndex, citiesInfo, countriesIndex, countriesInfo)
        inferred_countries = inferCountryFromCity(user_cities, citiesIndex, citiesInfo, ccDict)
        city, country = getFinalUserLocation(user_cities, user_countries, inferred_countries)
    else:
        city = tweet_place_city
        country = tweet_place_country

    if (tweet_place_country is not None) or (country is not None):
        hasCountry =True
        countPlace += 1

    # 4. extract date.day as int
    day = int(time.strftime('%Y%m%d', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')))

    # 5. extract tweet_id as long!
    tweet_id = tweet["id"]
    
    # SEE if CONDITIONS are met -> add to index
    if hasImg and hasCategory and hasCountry:
        tweet_dict = dict()
        tweet_dict["day"] = day
        tweet_dict["img_category"] = get_food_category(tweet)
        tweet_dict["city"] = city
        tweet_dict["country"] = country
        tweet_dict["created_at_datetime"] = datetime.strptime(tweet["created_at"], '%a %b %d %H:%M:%S +0000 %Y')
        tweet_dict["tweet_body"] = tweet
    
        es.index(index='trends', doc_type='tweet', id=tweet_id, body=tweet_dict)

print "Total relevant tweets in a day: ", i
print "Tweets with media", countImg
print "Tweets with category", countCategory
print "Tweets with place", countPlace

In [ ]:
# es.delete_by_query(index='trends', doc_type='tweet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

In [ ]:
# Search tweet text for QUERY
es.search(index="trends", body={"query": {"match": {'tweet_body.text':'lasagna'}}})

In [ ]:
# Search tweet for ID
es.get(index='trends', doc_type='tweet', id=855919564705673216)
# http://foodmap.isti.cnr.it:9200/trends/tweet/856023708892360704

In [ ]:
# dealing with the index 
r = requests.get('http://foodmap.isti.cnr.it:9200/',  auth=('elastic', 'changeme'))

In [6]:
from index.preprocess_tweet import *
from twitter.Tweet import Tweet


categoryList = loadCartegoryList()

tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i=0
for tweet in tweetsAsDict:
    print tweet["text"]
    print get_tweet_text_category(tweet, categoryList)
    
    i+=1
    if i%100==0:
        break

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251
Opening file:  /home/foodmap/food-tweets-2017-04-23.json.gz
Please RT #recipes #food #cooking #delicious #cook #recipe Perfectly Packable Lunches &amp;#124; Wicked... https://t.co/2dNkXG88DY
[]
Chelsea's Corner
Cafe &amp; Bar
Join us for Great Music and Great Food
https://t.co/eXMMboEhHE
#livemusic #pizza #bands https://t.co/UxXm8OWkKR
[u'#pizza']
Ven a #BistroPedregal, tenemos muchas opciones.
#Restaurante #Food #CDMX #Sábado https://t.co/q9ufZ0prti
[]
#foodie［  접속 https://t.co/QlaxPUzqLt ］［#성인고스톱］［#LLC］ #nickomalley ［#카ㅈㅣ노사이트］ ［#amor］［#월드카ㅈㅣ노］ ［#6］그래서 몇
[]
Stonewall Kitchen Breakfast Gift Basket @ https://t.co/vvIW0mFuan  #breakfast #jelly #jam #uniquegifts… https://t.co/XU3QmQ9sdn
[]
@Sushi_Murai +56971247174 752570634 #Curicó #sushi #amigos https://t.co/4OZizty4qt
[u'#sushi']
Got an awesome surprise goodie box 